<a href="https://colab.research.google.com/github/ashu5644/audio_workspace/blob/main/librosa_tutorial.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install librosa==0.10.0
import librosa

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 252.9/252.9 kB 16.5 MB/s eta 0:00:00
  Attempting uninstall: librosa
    Found existing installation: librosa 0.10.0.post2
    Uninstalling librosa-0.10.0.post2:
      Successfully uninstalled librosa-0.10.0.post2


**Audio Loading**

In [3]:
import numpy as np
!wget https://voiceage.com/wbsamples/in_mono/Conference.wav
y, sr = librosa.load('Conference.wav', sr=16000, mono=True, dtype=np.float32, offset=1, duration=1)
print(y.shape, sr)

--2023-05-24 06:04:59--  https://voiceage.com/wbsamples/in_mono/Conference.wav
Resolving voiceage.com (voiceage.com)... 24.37.80.233
Connecting to voiceage.com (voiceage.com)|24.37.80.233|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 472488 (461K) [audio/x-wav]
Saving to: ‘Conference.wav’

Conference.wav      100%[===================>] 461.41K  2.77MB/s    in 0.2s    

2023-05-24 06:04:59 (2.77 MB/s) - ‘Conference.wav’ saved [472488/472488]

(16000,) 16000


In [4]:
librosa.get_samplerate('Conference.wav')

48000

In [5]:
#librosa.stream

In [8]:
#!wget https://voiceage.com/wbsamples/in_stereo/Conference.wav
y, sr = librosa.load('Conference.wav.1', mono=False)
print(y.shape, sr)

(2, 113807) 22050


In [9]:
y_mono = librosa.to_mono(y)
print(y_mono.shape)

(113807,)


In [11]:
duration = librosa.get_duration(y=y_mono, sr=22050)
duration

5.161315192743764

In [12]:
sr = librosa.get_samplerate('Conference.wav.1')
sr

48000

In [13]:
y_16k = librosa.resample(y_mono, orig_sr=22050, target_sr=16000)
print(y_16k.shape)

(82582,)
